## Polytope Climate-DT example notebook

This notebook shows how to use earthkit-data and earthkit-plots to pull destination-earth data from LUMI and plot it using earthkit-plots. Specifically this notebook shows how to do server side interpolation to a supported grid.

Before running the notebook you need to set up your credentials. See the main readme of this repository for different ways to do this or use the following cells to authenticate.

You will need to generate your credentials using the desp-authentication.py script.

This can be run as follows:

In [ ]:
%%capture cap
%run ../desp-authentication.py

This will generate a token that can then be used by earthkit and polytope.

In [ ]:
output_1 = cap.stdout.split('}\n')
access_token = output_1[-1][0:-1]

# Requirements
To run this notebook install the following:
* pip install earthkit-data
* pip install earthkit-plots
* pip install earthkit-regrid  (Optional for spectral variables)
* pip install cf-units         (Optional for unit conversion in maps)

If you do not have eccodes installed please install eccodes using conda as it is a dependency of earthkit, or install earthkit via conda

* conda install eccodes -c conda-forge
* conda install earthkit-data -c conda-forge

In [ ]:
import earthkit.data
import earthkit.plots
import earthkit.regrid
from polytope.api import Client

In [ ]:

client = Client(
    address="polytope.lumi.apps.dte.destination-earth.eu")

request = {
    'activity': 'scenariomip',
    'class': 'd1',
    'dataset': 'climate-dt',
    'date': '20300101',
    'time': "0000",
    'experiment': 'SSP3-7.0',
    'expver': '0001',
    'generation': '1',
    'levtype': 'sfc',
    'model': 'IFS-NEMO',
    'param': '167',
    'realization': '1',
    'resolution': 'high',
    'stream': 'clte',
    'grid' : 'O2560', # currently O, F, N grids are supported 
    'type': 'fc',
}

file = client.retrieve("destination-earth", request, "data/climate-dt-earthkit-grid-example.grib")


In [ ]:
data = earthkit.data.from_source("file", "data/climate-dt-earthkit-grid-example.grib")

In [ ]:
data.ls()

In [ ]:
ds = data.to_xarray()
ds

In [ ]:
data_latlon = earthkit.regrid.interpolate(
    data,
    out_grid={"grid": [0.1,0.1]},
    method="linear"
)

In [ ]:
chart = earthkit.plots.Map()

chart.contourf(
    data_latlon,
    auto_style=True
)

chart.coastlines()
chart.borders()
chart.gridlines()
chart.legend()

chart.show()